In [10]:
# Just verify that Spark Session is available
spark

In [11]:
# Necessary PySparkling imports
from pysparkling import *
from pysparkling.ml import *

In [12]:
# Start H2O Context
hc = H2OContext.getOrCreate(spark)

Connecting to H2O server at http://10.20.24.129:54321... successful.


H2O cluster uptime:,2 mins 46 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.11
H2O cluster version age:,11 days
H2O cluster name:,sparkling-water-kuba_local-1528178693317
H2O cluster total nodes:,1
H2O cluster free memory:,2.291 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"



Sparkling Water Context:
 * H2O name: sparkling-water-kuba_local-1528178693317
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (driver,10.20.24.129,54321)
  ------------------------

  Open H2O Flow in browser: http://10.20.24.129:54321 (CMD + click in Mac OSX)

    


In [13]:
# Filter out the outliers
from pyspark.ml.feature import SQLTransformer
removeOutliers = SQLTransformer(
    statement="SELECT * FROM __THIS__ WHERE nrows_train > 100")

In [14]:
# Define the H2O AutoML pipeline stage for Spark pipeline
from pysparkling.ml import *
algoStage = H2OAutoML(
                   maxRuntimeSecs=60, # 1 minute
                   seed=1,
                   convertUnknownCategoricalLevelsToNa=True,
                   predictionCol="t_total")

In [15]:
# Define the Spark pipeline
from pyspark.ml import Pipeline, PipelineModel
pipeline = Pipeline(stages=[removeOutliers, algoStage])


In [16]:
# Imoprt the data
import h2o
fr = h2o.import_file("spark_ai_summit_2018.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
# Split data to training and testing dataset
splits = fr.split_frame(ratios=[0.8])
train = splits[0]
test = splits[1]

In [18]:
# Convert data to Spark frame from H2O frame
train_spark_frame = hc.as_spark_frame(train)
train_spark_frame

DataFrame[GPU: string, sys_CPU: tinyint, sys_GPU: tinyint, config_accu: tinyint, config_time: tinyint, config_MLI: tinyint, nrows_train: int, ncols_train: smallint, target_binary_dist: double, target_nclasses: smallint, recipe_individuals: tinyint, train_data_size: bigint, weight: tinyint, t_total: double]

In [19]:
# Train the pipeline model
model = pipeline.fit(train_spark_frame)

In [20]:
# Export the Pipeline Model
model.write().overwrite().save("sw.pipeline")